<a href="https://colab.research.google.com/github/niladri-rkmvu/dsa-2025/blob/10.sorting/sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All Sorting Algorithms

In [2]:
class Sorter:
    def __init__(self, A):
        # Initialize with a copy of the array
        self.A = list(A)

    def print_array(self):
        # Print array elements space-separated
        print(*self.A)

    def swap(self, i, j):
        # Swap two elements in the array
        self.A[i], self.A[j] = self.A[j], self.A[i]

    def reset(self, new_A):
        # Reset array to a new one
        self.A = list(new_A)

    # ---------------- Bubble Sort ----------------
    def bubble_sort(self):
        n = len(self.A)
        for i in range(n - 1):
            flag = False
            for j in range(n - i - 1):
                if self.A[j] > self.A[j + 1]:
                    self.swap(j, j + 1)
                    flag = True
            if not flag:
                break

    # ---------------- Insertion Sort ----------------
    def insertion_sort(self):
        n = len(self.A)
        for i in range(1, n):
            key = self.A[i]
            j = i - 1
            while j >= 0 and self.A[j] > key:
                self.A[j + 1] = self.A[j]
                j -= 1
            self.A[j + 1] = key

    # ---------------- Selection Sort ----------------
    def selection_sort(self):
        n = len(self.A)
        for i in range(n - 1):
            min_idx = i
            for j in range(i + 1, n):
                if self.A[j] < self.A[min_idx]:
                    min_idx = j
            self.swap(i, min_idx)

    # ---------------- Quick Sort ----------------
    def partition(self, l, h):
        pivot = self.A[l]
        i, j = l, h
        while i < j:
            while i <= h and self.A[i] <= pivot:
                i += 1
            while j >= l and self.A[j] > pivot:
                j -= 1
            if i < j:
                self.swap(i, j)
        self.swap(l, j)
        return j

    def quick_sort(self, l=0, h=None):
        if h is None:
            h = len(self.A) - 1
        if l < h:
            j = self.partition(l, h)
            self.quick_sort(l, j - 1)
            self.quick_sort(j + 1, h)

    # ---------------- Merge Sort ----------------
    def merge(self, l, mid, h):
        i, j = l, mid + 1
        temp = []

        while i <= mid and j <= h:
            if self.A[i] < self.A[j]:
                temp.append(self.A[i])
                i += 1
            else:
                temp.append(self.A[j])
                j += 1

        while i <= mid:
            temp.append(self.A[i])
            i += 1

        while j <= h:
            temp.append(self.A[j])
            j += 1

        for k in range(len(temp)):
            self.A[l + k] = temp[k]

    def merge_sort_iter(self):
        n = len(self.A)
        p = 1
        while p < n:
            for i in range(0, n - p, 2 * p):
                l = i
                mid = l + p - 1
                h = min(l + 2 * p - 1, n - 1)
                self.merge(l, mid, h)
            p *= 2

    def merge_sort_recursive(self, l=0, h=None):
        if h is None:
            h = len(self.A) - 1
        if l < h:
            mid = (l + h) // 2
            self.merge_sort_recursive(l, mid)
            self.merge_sort_recursive(mid + 1, h)
            self.merge(l, mid, h)


if __name__ == "__main__":
    A = [11, 13, 7, 12, 16, 9, 24, 5, 10, 3]
    sorter = Sorter(A)

    print("Unsorted Array:")
    sorter.print_array()

    # Bubble Sort
    sorter.reset(A)
    sorter.bubble_sort()
    print("Bubble Sorted Array:")
    sorter.print_array()

    # Insertion Sort
    sorter.reset(A)
    sorter.insertion_sort()
    print("Insertion Sorted Array:")
    sorter.print_array()

    # Selection Sort
    sorter.reset(A)
    sorter.selection_sort()
    print("Selection Sorted Array:")
    sorter.print_array()

    # Quick Sort
    sorter.reset(A)
    sorter.quick_sort()
    print("Quick Sorted Array:")
    sorter.print_array()

    # Merge Sort (Iterative)
    sorter.reset(A)
    sorter.merge_sort_iter()
    print("Merge Sorted Array (Iterative):")
    sorter.print_array()

    # Merge Sort (Recursive)
    sorter.reset(A)
    sorter.merge_sort_recursive()
    print("Merge Sorted Array (Recursive):")
    sorter.print_array()

Unsorted Array:
11 13 7 12 16 9 24 5 10 3
Bubble Sorted Array:
3 5 7 9 10 11 12 13 16 24
Insertion Sorted Array:
3 5 7 9 10 11 12 13 16 24
Selection Sorted Array:
3 5 7 9 10 11 12 13 16 24
Quick Sorted Array:
3 5 7 9 10 11 12 13 16 24
Merge Sorted Array (Iterative):
3 5 7 9 10 11 12 13 16 24
Merge Sorted Array (Recursive):
3 5 7 9 10 11 12 13 16 24
